In [1]:
import sys
import numpy as np
import math
import operator
import functools
import json
from pprint import pprint
from tqdm.notebook import tqdm, trange

In [73]:
class Instruction:
    def __init__(self, action, left, right=None):
        self.action = action
        self.left = left
        try:
            self.right = int(right)
        except Exception:
            self.right = right
        
    def getRightValue(self, vars):
        return self.right if type(self.right) == int else vars[self.right]
        
    def __repr__(self):
        if self.action == "inp":
            return f"{self.left} = digits.pop(0)"
        elif self.action == "add":
            return f"{self.left} += {self.right}"
        elif self.action == "mul":
            return f"{self.left} *= {self.right}"
        elif self.action == "div":
            return f"{self.left} = {self.left} // {self.right}"
        elif self.action == "mod":
            return f"{self.left} = {self.left} % {self.right}"
        elif self.action == "eql":
            return f"{self.left} = 1 if {self.left} == {self.right} else 0"
        return f"{self.action}({self.left}{',' if self.right else ''}{self.right if self.right else ''})"
    
    

class ALU:
    def __init__(self):
        self.instructions = []
        self.cache = {}
    
    def resetVars(self):
        self.vars = {
            "w": 0,
            "x": 0,
            "y": 0,
            "z": 0,
        }
    
    def addInstruction(self, instr_str):
        self.instructions.append(Instruction(*instr_str.split()))
    
    def isValid(self, num) -> bool:
        self.resetVars()
        num_lst = [int(i) for i in str(num)]
        # print(self.instructions)
        for i in self.instructions:
            # print(self.vars)
            if i.action == 'inp':
                self.vars[i.left] = num_lst.pop(0)
            elif i.action == "add":
                self.vars[i.left] += i.getRightValue(self.vars)
            elif i.action == "mul":
                self.vars[i.left] *= i.getRightValue(self.vars)
            elif i.action == "div":
                self.vars[i.left] = self.vars[i.left] // i.getRightValue(self.vars)
            elif i.action == "mod":
                self.vars[i.left] = self.vars[i.left] % i.getRightValue(self.vars)
            elif i.action == "eql":
                self.vars[i.left] = 1 if self.vars[i.left] == i.getRightValue(self.vars) else 0
            # print(i, self.vars)
        # print(self.vars)
        return True if self.vars["z"] == 0 else False
    
    def run(self):
        self.reached = 0
        return self._run(0, "")
    
    def _run(self, step, num):
        # if num == "99999999999529":
        #     self.reached = True
        # if self.reached:
        #     return num, False
        if len(num) > 0 and num in self.cache:
            _vars = self.cache[num]
            # print(num, _vars)
        else:
            _vars = {
                "w": 0,
                "x": 0,
                "y": 0,
                "z": 0,
            }
        i = self.instructions[step]
        # print(self.cache)
        # print(num, i, _vars)
        if i.action == 'inp':
            for x in range(9,0, -1):
                _vars[i.left] = x
                self.cache[num + str(x)] = _vars.copy()
                _num, valid = self._run(step+1, num + str(x))
                if valid:
                    return _num, valid
            else:
                return num, False
        elif i.action == "add":
            _vars[i.left] += i.getRightValue(_vars)
        elif i.action == "mul":
            _vars[i.left] *= i.getRightValue(_vars)
        elif i.action == "div":
            _vars[i.left] = _vars[i.left] // i.getRightValue(_vars)
        elif i.action == "mod":
            _vars[i.left] = _vars[i.left] % i.getRightValue(_vars)
        elif i.action == "eql":
            _vars[i.left] = 1 if _vars[i.left] == i.getRightValue(_vars) else 0
        
        if step == len(self.instructions)-1:
            # print(num, _vars)
            self.reached += 1
            if self.reached % 1000000 == 0:
                print(num)
            return num, _vars["z"] == 0
        else:
            self.cache[num] = _vars
            return self._run(step+1, num)
        
    def exportInstructions(self):
        with open("export.py", "w") as f:
            f.write("def execute(digits):\n")
            f.write("\tw = x = y = z = 0\n")
            for i in self.instructions:
                f.write("\t" + f"{i}" + "\n")
            f.write("\treturn z")
        

In [78]:
def part1(arr):
    alu = ALU()
    for i in arr:
        alu.addInstruction(i)
    largest = 95299897999897
    print(alu.isValid(largest), alu.vars)
    return largest

In [90]:
def part2(arr):
    alu = ALU()
    for i in arr:
        alu.addInstruction(i)
    smallest = 31111121382151
    print(alu.isValid(smallest), alu.vars)
    return smallest

In [91]:
if __name__ == "__main__":
    file = sys.argv[-1] if sys.argv[-1].endswith(".txt") else "input.txt"
    with open(file, "r") as f:
        arr = f.read().splitlines()

        print(part1(arr))
        print(part2(arr))

True {'w': 7, 'x': 0, 'y': 0, 'z': 0}
95299897999897
True {'w': 1, 'x': 0, 'y': 0, 'z': 0}
31111121382151


In [92]:
# This is the set of instructions in python
"""
for i in range(14):
    w = digits.pop(0)
    x *= 0
    x += z
    x = x % 26 
    z = z // {a} # 1, 1, 1, 1, 26, 1, 26, 1,26,26, 1,26,26,26
    x += {b}     # 11,14,10,14,-8,14,-11,10,-6,-9,12,-5,-4,-9
    x = 1 if x == w else 0
    x = 1 if x == 0 else 0
    y *= 0
    y += 25
    y *= x 
    y += 1
    z *= y
    y *= 0
    y += w
    y += {c} # 7,8,16,8,3,12,1,8,8,14,4,14,15,6
    y *= x
    z += y
"""
"""
Shortened
w = digits.pop(0)
x = int(z % 26 + b != w)
z = z // a
z *= (25*x)+1
z += (w+c)*x

if a == 1 then  10 < b < 14:
w = digits.pop(0)
z = z*26 + (w + c)

else then      -11 < b < -4
w = digits.pop(0)
x = int(z % 26 + b != w)
z = z // 26
z *= (25*x)+1
z += (w+c)*x

w_old+c == w_now-b
I[3]+8 == I[4]+8   => I[3] == I[4]
I[5]+12 == I[6]+11 => I[5]+1 == I[6]
I[7]+8 == I[8]+6   => I[7]+2 == I[8]
I[2]+16 == I[9]+9  => I[2]+7 == I[9]
I[10]+4 == I[11]+5 => I[10]-1 == I[11]
I[1]+8 == I[12]+4  => I[1]+4 == I[12]
I[0]+7 == I[13]+9  => I[0]-2 == I[13]
    01234567890123
I = 95299897999897
I = 31111121382151
"""

'\nShortened\nw = digits.pop(0)\nx = int(z % 26 + b != w)\nz = z // a\nz *= (25*x)+1\nz += (w+c)*x\n\nif a == 1 then  10 < b < 14:\nw = digits.pop(0)\nz = z*26 + (w + c)\n\nelse then      -11 < b < -4\nw = digits.pop(0)\nx = int(z % 26 + b != w)\nz = z // 26\nz *= (25*x)+1\nz += (w+c)*x\n\nw_old+c == w_now-b\nI[3]+8 == I[4]+8   => I[3] == I[4]\nI[5]+12 == I[6]+11 => I[5]+1 == I[6]\nI[7]+8 == I[8]+6   => I[7]+2 == I[8]\nI[2]+16 == I[9]+9  => I[2]+7 == I[9]\nI[10]+4 == I[11]+5 => I[10]-1 == I[11]\nI[1]+8 == I[12]+4  => I[1]+4 == I[12]\nI[0]+7 == I[13]+9  => I[0]-2 == I[13]\n    01234567890123\nI = 95299897999897\nI = 31111121382151\n'

8